In [77]:
import os
import pandas as pd

In [65]:
prj = '../../output/genome-wf'

In [139]:
dfi = pd.read_csv('../../output/genome-wf/genome_cov.bedgraph', sep='\t', chunksize=100000, 
                  header=None, index_col=[0, 1, 2])

dfs = []
for df in dfi:
    dfs.append(df[df.sum(axis=1) == 0].reset_index().iloc[:, :3])

df = pd.concat(dfs)
df.columns = ['chrom', 'start', 'end']
df.set_index('chrom', inplace=True)
df['count'] = df.end - df.start

In [140]:
df.head()

,start,end,count
chrom,,,
chr2L,0.0,4967.0,4967.0
chr2L,20947.0,20988.0,41.0
chr2L,47668.0,48264.0,596.0
chr2L,48563.0,49683.0,1120.0
chr2L,49805.0,50226.0,421.0


In [163]:
totals = df.groupby(level=0).apply(lambda x: x['count'].sum()).to_frame()
totals.columns = ['total']
totals.head()

,total
chrom,
chr2L,943933.0
chr2R,1393006.0
chr3L,1485256.0
chr3R,1236090.0
chr4,61343.0


In [166]:
sizes = pd.read_csv('../../output/dm6.chrom.sizes', header=None, sep='\t', names=['chrom', 'len'], index_col=0)
sizes.head()

,len
chrom,
chr3R,32079331
chr3L,28110227
chr2R,25286936
chrX,23542271
chr2L,23513712


In [167]:
merged = sizes.join(totals)

In [170]:
merged.head()

,len,total
chrom,,
chr3R,32079331,1236090.0
chr3L,28110227,1485256.0
chr2R,25286936,1393006.0
chrX,23542271,1146321.0
chr2L,23513712,943933.0


In [173]:
effective_chrom_size = (merged.len - merged.total).to_frame()

In [175]:
effective_chrom_size.sum()

0    133997162.0
dtype: float64

In [176]:
CHROMS = [
    'chr2L', 
    'chr2R',
    'chr3L', 
    'chr3R',
    'chr4', 
    'chrX',
    'chrY',
]

In [185]:
dm6 = effective_chrom_size[effective_chrom_size.index.isin(CHROMS)].sum().values[0]

In [180]:
dm3 = 121400000 

In [188]:
print('{:,}'.format(dm6))

129,848,422.0
